# Pipeline for training and evaluating a CNN model for traffic sign recognition
Notebook contains the following steps:
1. Data loading and preprocessing
2. Model definition
3. Training and evaluation
4. Saving results and model to Google Cloud Storage
5. Hyperparameter tuning


In [1]:
# required for logging
!pip install tensorboard

  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
Using cached Markdown-3.6-py3-none-any.whl (105 kB)
Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl (6.6 MB)
Using cached werkzeug-3.0.3-py3-none-any.whl (227 kB)


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, DataLoader, Subset, ConcatDataset
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix
from google.cloud import storage
import random
import math
from torch.optim.lr_scheduler import StepLR
import datetime

In [3]:
def set_seed(seed=42):
    """Sets the seed for reproducibility."""
    # Python RNG
    random.seed(seed)
    
    # PyTorch RNGs
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
    # Numpy RNG
    np.random.seed(seed)
    
    # OS RNG
    os.environ['PYTHONHASHSEED'] = str(seed)

def worker_init_fn(worker_id):    
    """Ensure that the data loading process is deterministic."""
    np.random.seed(np.random.get_state()[1][0] + worker_id)

set_seed(42)

In [4]:
project_id = 'deep-learning-420208'

In [5]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Data loading and preprocessing
AugmentationTrafficSignLoader class is used to load the augmented datasets and calculate the mean and standard deviation of the training data. Can load multiple augmentation datasets and calculate the mean and standard deviation for each dataset.

In [6]:
class AugmentationTrafficSignLoader:
    def __init__(self, root):
        self.root = root
        self.augmentations = [folder for folder in os.listdir(root)
                              if os.path.isdir(os.path.join(root, folder))]

    def calculate_mean_and_variance(self, training_root, percentage_of_whole):
    
        transform = transforms.Compose([
                transforms.Resize((96, 96)),
                transforms.ToTensor()
            ])

        test_dataset = datasets.ImageFolder(root=training_root, transform=transform)

        indices = random.sample(population=list(range(len(test_dataset))), k=math.floor(len(test_dataset)*percentage_of_whole))
        sample = Subset(test_dataset, indices)
        loader = DataLoader(sample)

        mean = 0.0
        variance = 0.0
        total_images = 0

        for images, _ in loader:
            # Rearrange batch to be the shape of [B, C, W * H]
            images = images.view(images.size(0), images.size(1), -1)
            # Update total_images
            total_images += images.size(0)
            # Compute mean and variance here
            mean += images.mean(2).sum(0) 
            variance += images.var(2).sum(0)

        # Final mean and variance
        mean /= total_images
        variance /= total_images

        return mean, variance.sqrt()
    
    def augmentation_generator(self):
        while self.augmentations:
            current_aug = self.augmentations.pop()
            current_aug_path = os.path.join(self.root, current_aug)
            mean, std = self.calculate_mean_and_variance(current_aug_path, 0.25)
            #print(mean, std)
            transform = transforms.Compose([
                transforms.Resize((96, 96)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std)
            ])
            yield os.path.basename(current_aug), datasets.ImageFolder(root=current_aug_path, transform=transform), mean, std

## Model definition
The CNN1 class defines the architecture of the convolutional neural network. The model consists of three convolutional layers followed by max pooling and batch normalization. The convolutional layers are followed by three fully connected layers.

In [7]:
class CNN1(nn.Module):
    def __init__(self, n_classes=0):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=100, kernel_size=3, padding=1, bias=False)  # Output size: 96 x 96 x 100
        self.norm1 = nn.BatchNorm2d(num_features=100)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output size: 48 x 48 x 100
        self.conv2 = nn.Conv2d(in_channels=100, out_channels=150, kernel_size=3, padding=1, bias=False)  # Output size: 48 x 48 x 150
        self.norm2 = nn.BatchNorm2d(num_features=150)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output size: 24 x 24 x 150
        self.conv3 = nn.Conv2d(in_channels=150, out_channels=250, kernel_size=3, padding=1, bias=False)  # Output size: 24 x 24 x 250
        self.norm3 = nn.BatchNorm2d(num_features=250)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output size: 12 x 12 x 250
        # Calculate the total number of features in the flattened feature map
        flat_features = 12 * 12 * 250
        self.fc1 = nn.Linear(in_features=flat_features, out_features=4000)  # Output: 4000 units
        self.fc2 = nn.Linear(in_features=4000, out_features=200)  # Output: 200 units
        self.fc3 = nn.Linear(in_features=200, out_features=n_classes)  #33 Output: 46 units

    def forward(self, x):
        x = self.pool1(self.norm1(F.relu(self.conv1(x))))
        x = self.pool2(self.norm2(F.relu(self.conv2(x))))
        x = self.pool3(self.norm3(F.relu(self.conv3(x))))
        x = torch.flatten(x, start_dim=1)  # Flatten starting from dimension 1
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Helper functions

In [8]:
def upload_directory_to_gcs(bucket_name, source_directory, destination_blob_prefix):
    """Uploads a local directory and its subdirectories to a GCS bucket."""
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket_name)

    for dirpath, dirnames, filenames in os.walk(source_directory):
        for filename in filenames:
            local_file_path = os.path.join(dirpath, filename)
            relative_path = os.path.relpath(local_file_path, source_directory)
            blob_path = os.path.join(destination_blob_prefix, relative_path)
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_file_path)
            #print(f"File {local_file_path} uploaded to {blob_path}.")

In [9]:
def calculate_accuracy(model, data_loader, device):
    """Calculates accuracy on given dataset"""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def evaluate_and_save_results(model, loader, device, classes, bucket_name, prefix, aug_name):
    """Evaluates the model and saves results and the model itself to Google Cloud Storage under a specific subfolder."""
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute the confusion matrix
    cm = confusion_matrix(all_labels, all_preds, labels=np.arange(len(classes)))

    # Folder prefix including augmentation name
    full_prefix = f"{prefix}/results/{aug_name}"
    os.makedirs(full_prefix)

    # Save confusion matrix
    cm_path = f"{full_prefix}/confusion_matrix_{aug_name}.npy"
    np.save(cm_path, cm)

    # Save predictions
    preds_path = f"{full_prefix}/predictions_{aug_name}.npy"
    np.save(preds_path, np.array(all_preds))

    # Save the model
    model_path = f"{full_prefix}/model_{aug_name}.pth"
    torch.save(model.state_dict(), model_path)

    return cm

def train_and_evaluate(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, device, writer, prefix, epochs=5, patience=10, min_delta=0.001):
    best_val_loss = float('inf')
    best_val_acc = 0
    epochs_no_improve = 0
    early_stop = False
    # Path to save the best model
    os.makedirs(prefix)
    best_model_path = f'{prefix}/best_model.pth'

    for epoch in tqdm(range(epochs)):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        scheduler.step()  # Update the learning rate    

        train_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
        val_loss /= len(val_loader.dataset)

        train_accuracy = calculate_accuracy(model, train_loader, device)
        val_accuracy = calculate_accuracy(model, val_loader, device)
        test_accuracy = calculate_accuracy(model, test_loader, device)

        writer.add_scalars('Loss', {'Train': train_loss, 'Validation': val_loss}, epoch)
        writer.add_scalars('Accuracy', {'Train': train_accuracy, 'Validation': val_accuracy}, epoch)

        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)  # Save the best model checkpoint
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                early_stop = True
                break

        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%, Test Acc: {test_accuracy:.2f}%')

    model.load_state_dict(torch.load(best_model_path, map_location=device))
    test_accuracy = calculate_accuracy(model, test_loader, device)
    print(test_accuracy)
    writer.add_scalar('Accuracy/test', test_accuracy, 1)
    writer.close()
    return test_accuracy


# Training and evaluation loop
This loop trains and evaluates the model for each augmentation dataset. The results are saved to Google Cloud Storage.

In [ ]:
runs = [
    ['data/synthetic', 'data/test_data']
]
for TRAINING_DATA_ROOT, TEST_DATA_ROOT in runs:
    BUCKET_NAME = 'sign-recognition-metrics'
    PREFIX = 'metrics/' + datetime.now().strftime('%Y%m%d-%H%M%S')

    train_augmentation_generator = AugmentationTrafficSignLoader(root=TRAINING_DATA_ROOT).augmentation_generator()
    for aug_name, aug_variant, mean, std in train_augmentation_generator:
        # Ensure GPU memory is clean before starting the setup
        torch.cuda.empty_cache()
        print(aug_name)

        # Data loading setup
        num_classes = len(aug_variant.classes)
        #print(num_classes)
        #Model initialization and setup
        model = CNN1(num_classes)
        model = nn.DataParallel(model)
        model.to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.001) #, weight_decay=0.005)
        # Setup scheduler
        scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Adjusts the learning rate every 10 epochs by halving it
        criterion = nn.CrossEntropyLoss()


        #print(aug_variant.classes)
        train_size = int(0.8 * len(aug_variant))
        val_size = len(aug_variant) - train_size
        train_dataset, val_dataset = random_split(aug_variant, [train_size, val_size])
        train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=16, worker_init_fn=worker_init_fn)
        val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)

        # Load the test dataset
        transform = transforms.Compose([
                    transforms.Resize((96, 96)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=mean, std=std)
                ])
        test_dataset = datasets.ImageFolder(root=TEST_DATA_ROOT, transform=transform)
        idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)
        #print(mean, std)

        # TensorBoard Setup
        writer_dir = os.path.join(PREFIX, 'runs', aug_name)
        writer = SummaryWriter(writer_dir)

        model_dir = f"{PREFIX}/models/{aug_name}"
        # Training and Evaluation
        test_accuracy = train_and_evaluate(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, device, writer, model_dir, epochs=40)
        cm = evaluate_and_save_results(model, test_loader, device, idx_to_class, BUCKET_NAME, PREFIX, aug_name)

        # Upload results
        upload_directory_to_gcs(BUCKET_NAME, PREFIX, PREFIX)

        # Clear memory
        torch.cuda.empty_cache()

8


  2%|▎         | 1/40 [03:50<2:29:42, 230.31s/it]

Epoch 1, Train Loss: 1.0730, Val Loss: 0.1695, Train Acc: 94.73%, Val Acc: 94.04%, Test Acc: 67.25%


  5%|▌         | 2/40 [05:27<1:36:19, 152.08s/it]

Epoch 2, Train Loss: 0.0446, Val Loss: 0.0809, Train Acc: 97.87%, Val Acc: 97.60%, Test Acc: 68.63%


...

 32%|███▎      | 13/40 [22:06<41:39, 92.59s/it]

Epoch 13, Train Loss: 0.0000, Val Loss: 0.0004, Train Acc: 100.00%, Val Acc: 99.97%, Test Acc: 41.32%


 32%|███▎      | 13/40 [23:38<49:06, 109.12s/it]

Early stopping triggered after 14 epochs.


45.601851851851855


## Batch size hyperparameter tuning

In [ ]:
batches = [512, 256, 128, 64, 32, 16]
TRAINING_DATA_ROOT = 'data/synthetic/8'
TEST_DATA_ROOT = 'data/test_data'


BUCKET_NAME = 'sign-recognition-metrics'
PREFIX = 'metrics/batch_' + datetime.now().strftime('%Y%m%d-%H%M%S')
best_batch_accuracy = 0
best_batch_size = 512

train_augmentation_generator = AugmentationTrafficSignLoader(root=TRAINING_DATA_ROOT).augmentation_generator()
for aug_name, aug_variant, mean, std in train_augmentation_generator:
    for BATCH_SIZE in batches:
        print(BATCH_SIZE)
        # Ensure GPU memory is clean before starting the setup
        torch.cuda.empty_cache()
        print(aug_name)

        # Data loading setup
        num_classes = len(aug_variant.classes)
        #print(num_classes)
        #Model initialization and setup
        model = CNN1(num_classes)
        model = nn.DataParallel(model)
        model.to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.001) #, weight_decay=0.005)
        # Setup scheduler
        scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Adjusts the learning rate every 10 epochs by halving it
        criterion = nn.CrossEntropyLoss()


        #print(aug_variant.classes)
        train_size = int(0.8 * len(aug_variant))
        val_size = len(aug_variant) - train_size
        train_dataset, val_dataset = random_split(aug_variant, [train_size, val_size])
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=16, worker_init_fn=worker_init_fn)
        val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)

        # Load the test dataset
        transform = transforms.Compose([
                    transforms.Resize((96, 96)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=mean, std=std)
                ])
        test_dataset = datasets.ImageFolder(root=TEST_DATA_ROOT, transform=transform)
        idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)
        #print(mean, std)

        # TensorBoard Setup
        writer_dir = os.path.join(PREFIX, 'runs', f'{BATCH_SIZE}')
        writer = SummaryWriter(writer_dir)

        model_dir = f"{PREFIX}/models/{BATCH_SIZE}"
        # Training and Evaluation
        test_accuracy = train_and_evaluate(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, device, writer, model_dir, epochs=40)
        if test_accuracy > best_batch_accuracy:
            best_batch_accuracy = test_accuracy
            best_batch_size = BATCH_SIZE
        cm = evaluate_and_save_results(model, test_loader, device, idx_to_class, BUCKET_NAME, PREFIX, aug_name)

        # Upload results
        upload_directory_to_gcs(BUCKET_NAME, PREFIX, PREFIX)

        # Clear memory
        torch.cuda.empty_cache()

## Dropout and L2 regularization hyperparameter tuning
With best batch size found in the previous step, we will now tune the dropout hyperparameters. Uncomment decay in the optimizer initialization to enable L2 regularization.

In [ ]:
class CNN1(nn.Module):
    def __init__(self, n_classes=35):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=100, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.BatchNorm2d(num_features=100)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)  # Dropout after first pooling

        self.conv2 = nn.Conv2d(in_channels=100, out_channels=150, kernel_size=3, padding=1, bias=False)
        self.norm2 = nn.BatchNorm2d(num_features=150)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)  # Dropout after second pooling

        self.conv3 = nn.Conv2d(in_channels=150, out_channels=250, kernel_size=3, padding=1, bias=False)
        self.norm3 = nn.BatchNorm2d(num_features=250)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.5)  # Dropout after third pooling

        flat_features = 12 * 12 * 250
        self.fc1 = nn.Linear(in_features=flat_features, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=n_classes)
        self.dropout4 = nn.Dropout(0.8)  # Dropout before the fully connected layers

    def forward(self, x):
        x = self.dropout1(self.pool1(self.norm1(F.relu(self.conv1(x)))))
        x = self.dropout2(self.pool2(self.norm2(F.relu(self.conv2(x)))))
        x = self.dropout3(self.pool3(self.norm3(F.relu(self.conv3(x)))))
        x = torch.flatten(x, start_dim=1)
        x = self.dropout4(F.relu(self.fc1(x)))
        x = self.dropout4(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
BUCKET_NAME = 'sign-recognition-metrics'
PREFIX = 'metrics/drop_' + datetime.now().strftime('%Y%m%d-%H%M%S')
train_augmentation_generator = AugmentationTrafficSignLoader(root=TRAINING_DATA_ROOT).augmentation_generator()
for aug_name, aug_variant, mean, std in train_augmentation_generator:
    # Ensure GPU memory is clean before starting the setup
    torch.cuda.empty_cache()
    print(aug_name)

    # Data loading setup
    num_classes = len(aug_variant.classes)
    #print(num_classes)
    #Model initialization and setup
    model = CNN1(num_classes)
    model = nn.DataParallel(model)
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001) #, weight_decay=0.005)
    # Setup scheduler
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Adjusts the learning rate every 10 epochs by halving it
    criterion = nn.CrossEntropyLoss()


    #print(aug_variant.classes)
    train_size = int(0.8 * len(aug_variant))
    val_size = len(aug_variant) - train_size
    train_dataset, val_dataset = random_split(aug_variant, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True, num_workers=16, worker_init_fn=worker_init_fn)
    val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)

    # Load the test dataset
    transform = transforms.Compose([
                transforms.Resize((96, 96)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std)
            ])
    test_dataset = datasets.ImageFolder(root=TEST_DATA_ROOT, transform=transform)
    idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=16, worker_init_fn=worker_init_fn)
    #print(mean, std)

    # TensorBoard Setup
    writer_dir = os.path.join(PREFIX, 'runs', aug_name)
    writer = SummaryWriter(writer_dir)

    model_dir = f"{PREFIX}/models/{aug_name}"
    # Training and Evaluation
    test_accuracy = train_and_evaluate(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, device, writer, model_dir, epochs=40)
    if test_accuracy > best_batch_accuracy:
        best_batch_accuracy = test_accuracy
        best_batch_size = BATCH_SIZE
    cm = evaluate_and_save_results(model, test_loader, device, idx_to_class, BUCKET_NAME, PREFIX, aug_name)

    # Upload results
    upload_directory_to_gcs(BUCKET_NAME, PREFIX, PREFIX)

    # Clear memory
    torch.cuda.empty_cache()